In [0]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql import *

### Question: Find the names of the customer that are not referred by the customer with id = 2.

In [0]:
leet_code_data = [
    (1, 'Will', None),
    (2, 'Jane', None),
    (3, 'Alex', 2),
    (4, 'Bill', None),
    (5, 'Zack', 1),
    (6, 'Mark', 2)
]

schema = ["id","name","referee_id"]

customer_df = spark.createDataFrame(leet_code_data, schema)

customer_df.show()

**Using Dataframe**

**PROCESS--1: \
Step1: Get the name who has referee_id = 2 and store as a new dataframe \
Step2: Join the older and newer dataframe using leftanti join**

In [0]:
referee_2 = customer_df.select(col("name")).filter(col("referee_id") == 2)

customer_df.select(col("name")).join(referee_2,on="name",how="leftanti").show()

**PROCESS--2: \
Step1: Create a new column using withColumn where "Yes" if referee_id = 2 otherwise "No" \
Step2: Filter on top of that new column**

In [0]:
customer_df.withColumn("test", when(col("referee_id") == 2, "Yes")
                               .otherwise("No")) \
           .filter(col("test") == "No") \
           .select(col("name")) \
           .show()

**Using Spark SQL**

In [0]:
customer_df.createOrReplaceTempView("customer_table")

In [0]:
%sql
select name from customer_table where name not in (select name from customer_table where referee_id = 2)

### Question: Write a solution to report the first name, last name, city, and state of each person in the Person table. If the address of a personId is not present in the Address table, report null instead.

In [0]:
persons_data = [(1,'Wang','Allen'),
                (2,'Alice','Bob')]

persons_schema = ["personId","lastName","firstName"]

persons_df = spark.createDataFrame(persons_data,persons_schema)

persons_df.show()

address_data = [(1,2,'New York City','New York'),
                (2,3,'Leetcode','California')]

address_schema = ["addressId","personId","city","state"]

address_df = spark.createDataFrame(address_data,address_schema)

address_df.show()

**Using DataFrame**

In [0]:
report_df = persons_df.join(address_df, persons_df["personId"] == address_df["personId"], how="left") \
                      .select(persons_df["firstName"], persons_df["lastName"], address_df["city"], address_df["state"]).show()

**Using Spark SQL**

In [0]:
persons_df.createOrReplaceTempView("Persons_table")
address_df.createOrReplaceTempView("Address_table")

In [0]:
%sql
select P.firstName,P.lastName,A.city,A.state
from Persons_table P
left outer join Address_table A
on P.personId = A.personId;